In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.regularizers import l1
from tqdm import tqdm

In [9]:
# List all available devices detected by TensorFlow
print("Available devices:")
devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


## Load and prepare data

In [10]:
# Load the data
data = pd.read_csv("../dataset.csv")

In [11]:
# Separate features and target
X = data.drop("Label", axis=1)
y = data["Label"]

In [12]:
# Convert categorical data to numeric if necessary
X = pd.get_dummies(X, sparse=True)

In [13]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1059: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  if np.may_share_memory(array, array_orig):
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy arr

## Build an AutoEncoder

In [15]:
# Define the encoder with sparsity regularization
input_layer = Input(shape=(X_train_scaled.shape[1],))
encoded = Dense(128, activation="relu")(input_layer)

# Add L1 activity regularizer to the next dense layer
encoded = Dense(64, activation="relu", activity_regularizer=l1(10e-5))(encoded)

In [16]:
# Define the decoder
decoded = Dense(128, activation="relu")(encoded)
decoded = Dense(X_train_scaled.shape[1], activation="sigmoid")(decoded)

In [17]:
# Define the autoencoder
autoencoder = Model(input_layer, decoded)

# Encoder for predictions
encoder = Model(input_layer, encoded)

# Prediction model
predictor = Dense(1, activation="sigmoid")(encoded)
prediction_model = Model(input_layer, predictor)

# Compile models
autoencoder.compile(optimizer="adam", loss="mse")
prediction_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [18]:
# Display the model summary
autoencoder.summary()
prediction_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20090)          │     2,591,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,179,834 (19.76 MB)

 Trainable params: 5,179,834 (19.76 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,579,969 (9.84 MB)

 Trainable params: 2,579,969 (9.84 MB)

 Non-trainable params: 0 (0.00 B)

## Train and evaluate

In [19]:
# Train the autoencoder
autoencoder.fit(
    X_train_scaled, X_train_scaled, epochs=30, batch_size=256, validation_split=0.2
)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 1.4671 - val_loss: 1.0952
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 1.0319 - val_loss: 0.9892
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.9608 - val_loss: 0.9414
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.9109 - val_loss: 0.9123
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.8773 - val_loss: 0.8963
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.8577 - val_loss: 0.8843
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.8342 - val_loss: 0.8765
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.8186 - val_loss: 0.8736
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.8105 - val_loss: 0.8724
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.8069 - val_loss: 0.8719
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.8051 - val_loss: 0.8717
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.8

In [20]:
# Train the prediction model
prediction_model.fit(
    X_train_scaled, y_train, epochs=30, batch_size=256, validation_split=0.2
)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4988 - loss: 0.6932 - val_accuracy: 0.5081 - val_loss: 0.7777
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5153 - loss: 0.6931 - val_accuracy: 0.5081 - val_loss: 0.7777
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4726 - loss: 0.6932 - val_accuracy: 0.5088 - val_loss: 0.7778
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4946 - loss: 0.6932 - val_accuracy: 0.5075 - val_loss: 0.7779
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.4986 - loss: 0.6932 - val_accuracy: 0.5088 - val_loss: 0.7778
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5073 - loss: 0.6931 - val_accuracy: 0.5081 - val_loss: 0.7777
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5027 - loss: 0.6931 - val_accuracy: 0.5081 - val_loss: 0.7778
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5016 - loss: 0.6931 - val_accuracy: 0.5088 - v

In [21]:
# Evaluate the prediction model
test_loss, test_acc = prediction_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy: {:.2f}%".format(test_acc * 100))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4809 - loss: 0.6932
Test Accuracy: 49.50%
